In [ ]:
import pandas as pd 
from matplotlib import pyplot as plt
import numpy as np
import os

In [ ]:
os.getcwd()
os.chdir('/content/drive/MyDrive/REDD')

Import data:

In [ ]:


fr = pd.read_csv('channel_5.dat', header=None, sep=' ', engine='python', names=['a', 'b'])
fr = fr.b
ov = pd.read_csv('channel_3.dat', header=None, sep=' ', engine='python', names=['a', 'b'])
ov = ov.b 
dw = pd.read_csv('channel_6.dat', header=None, sep=' ', engine='python', names=['a', 'b'])
dw = dw.b
kit = pd.read_csv('channel_8.dat', header=None, sep=' ', engine='python', names=['a', 'b'])
kit = kit.b
wd = pd.read_csv('channel_10.dat', header=None, sep=' ', engine='python', names=['a', 'b'])
wd = wd.b
mic = pd.read_csv('channel_11.dat', header=None, sep=' ', engine='python', names=['a', 'b'])
mic = mic.b 

Create aggregate vector:

In [ ]:
agg = fr + ov + dw + kit + wd + mic 
# reshape 
agg = agg.to_numpy()
X = agg.reshape((len(agg),1))

Label data:

In [ ]:
thr =50

def labeling(data):
    labels =[]
    for i in range(0,len(data)):
        if (data[i]>thr):
            labels.append(1)
        else:
            labels.append(0)
            
    return np.array(labels) 
#%%
y_fr = labeling (fr)
y_fr = y_fr.reshape((len(y_fr),1))

y_ov = labeling (ov)
y_ov = y_ov.reshape((len(y_ov),1))

y_dw = labeling (dw)
y_dw = y_dw.reshape((len(y_dw),1))

y_kit = labeling (kit)
y_kit = y_kit.reshape((len(y_kit),1))

y_wd = labeling (wd)
y_wd = y_wd.reshape((len(y_wd),1))

y_mic = labeling (mic)
y_mic = y_mic.reshape((len(y_mic),1))

Create label matrix:

In [ ]:
Y = np.concatenate((y_fr,y_ov,y_dw,y_kit,y_wd,y_mic), axis=1)



---


STATE ESTIMATION USING MULTILABEL CLASSIFICATION :


---



Split data into train/test sets:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    test_size=0.2)

Helper functions:

In [ ]:
def train(model):
    model.fit(X_train, y_train)
    

from sklearn.metrics import f1_score

def evaluate():
    
    y_pred_train = model.predict(X_train)
    
    micro_f1 = f1_score(y_train, y_pred_train>0.5, average='micro')
    macro_f1 = f1_score(y_train, y_pred_train>0.5, average='macro') 

    print("training performance")
    print("micro-F1: %s"%micro_f1)
    print("macro-F1: %s"%macro_f1)
    # print("training error: %s"%error_training)
    print("")

    y_pred_test = model.predict(X_test)
        
    micro_f1 = f1_score(y_test, y_pred_test>0.5, average='micro')
                         
    macro_f1 = f1_score(y_test, y_pred_test>0.5, average='macro') 
    
    print("testing performance")
    print("micro-F1: %s"%micro_f1)
    print("macro-F1: %s"%macro_f1)
    # print("training error: %s"%error_training)
    print("")

Train and evaluate the classfier:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=0)

train(model)
evaluate()

training performance
micro-F1: 0.9221090962849995
macro-F1: 0.8566080314924207

testing performance
micro-F1: 0.9175432327030344
macro-F1: 0.8253331774663012



Try to improve performance by reducing variance in the aggregate signal:

In [ ]:
""" if the next agg sample doesn't vary by an amount greater
than thr_p, then copy the current agg sample """ 
thr_p=10
agg_stable = np.copy(agg)
 
for i in range (0, len(agg)-1):
    if (abs(agg_stable[i]-agg_stable[i+1])<thr_p):
        agg_stable[i+1] = agg_stable[i]
    else:
         pass

""" train with agg_stable """
X = agg_stable.reshape((len(agg_stable),1))
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    test_size=0.2,random_state=1)

train(model)
evaluate()

training performance
micro-F1: 0.9250315782508082
macro-F1: 0.8825350793100041

testing performance
micro-F1: 0.9188659432971648
macro-F1: 0.854839093793715



The performance improves slightly.



Compute metrcis:



In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
y_pred_test = model.predict(X_test)
conf_mtrx = multilabel_confusion_matrix(y_test, y_pred_test)

TP_fr = conf_mtrx[0][0][0]
FN_fr = conf_mtrx[0][0][1]
FP_fr = conf_mtrx[0][1][0]
TN_fr = conf_mtrx[0][1][1]

from sklearn.metrics import accuracy_score
# fr + ov + dw + kit + wd + mic

fr_acc = accuracy_score(y_test[:,0],y_pred_test[:,0])
ov_acc = accuracy_score(y_test[:,1],y_pred_test[:,1])
dw_acc = accuracy_score(y_test[:,2],y_pred_test[:,2])
kit_acc = accuracy_score(y_test[:,3],y_pred_test[:,3])
wd_acc = accuracy_score(y_test[:,4],y_pred_test[:,4])
mic_acc = accuracy_score(y_test[:,5],y_pred_test[:,5])

average_acc = np.average((fr_acc,ov_acc,dw_acc,kit_acc,wd_acc,mic_acc))

print("accuracies\n")

print("fr:{}".format(fr_acc))
print("ov:{}".format(ov_acc))
print("dw:{}".format(dw_acc))
print("kit:{}".format(kit_acc))
print("wd:{}".format(wd_acc))
print("mic:{}".format(mic_acc))

accuracies

fr:0.9840322840135143
ov:0.9991486566203679
dw:0.9857215637904221
kit:0.9672199281385746
wd:0.9963465973078779
mic:0.9976068536493806




---

POWER ESTIMATION USING MULTIOUTPUT REGRESSION :


---


The regression model is train/tested with Xe =[X|Y] as input and
Ye = [fr,ov,dw,kit,wd,mic] as outputs.

In [ ]:
Xe = np.concatenate((X,Y), axis=1)

fr = fr.to_numpy().reshape(-1,1)
ov = ov.to_numpy().reshape(-1,1)
dw = dw.to_numpy().reshape(-1,1)
kit = kit.to_numpy().reshape(-1,1)
wd = wd.to_numpy().reshape(-1,1)
mic = mic.to_numpy().reshape(-1,1)

Ye = np.concatenate((fr,ov,dw,kit,wd,mic), axis=1)

Split data:

In [ ]:
Xe_train, Xe_test, ye_train, ye_test = train_test_split(Xe, Ye,
                                                    test_size=0.2,random_state=1)

Train/test the regression model:

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

""" training and testing the energy estimator"""
clf = MultiOutputRegressor(Ridge(random_state=123)).fit(Xe_train
                                                      , ye_train)
""" predict on training data """
y_energy_pred_train = clf.predict(Xe_train)

""" predict on testing data """
y_energy_pred_test = clf.predict(Xe_test)
""" testing with the output of the classifier """

""" estimate and compare with ye_test. We first combine
X_test with the predictions of the classifer (y_pred_test)"""
cassifier_out = np.concatenate((X_test,y_pred_test), axis=1)

""" use the regressor to predict the power sample for each
load which is a reconstitution of the signal of each load"""
ener_est = clf.predict(cassifier_out)


Compute metrics:

In [ ]:
""" compute the relative energy estimation error for each laod """
error_fr = abs(np.sum(ye_test[:,0])- 
                   np.sum(ener_est[:,0]))/np.sum(ye_test[:,0])
error_ov = abs(np.sum(ye_test[:,1])- 
                   np.sum(ener_est[:,1]))/np.sum(ye_test[:,0])
error_dw = abs(np.sum(ye_test[:,2])- 
                   np.sum(ener_est[:,2]))/np.sum(ye_test[:,0])
error_kit = abs(np.sum(ye_test[:,3])- 
                   np.sum(ener_est[:,3]))/np.sum(ye_test[:,0])
error_wd = abs(np.sum(ye_test[:,4])- 
                   np.sum(ener_est[:,4]))/np.sum(ye_test[:,0])
error_mic = abs(np.sum(ye_test[:,5])- 
                   np.sum(ener_est[:,5]))/np.sum(ye_test[:,0])

average_error = np.average((error_fr,error_ov,error_dw,error_kit,error_wd,error_mic))
print("average error :{}".format(average_error))

""" compute NRMSE """
# reconstruct the predicted aggregate 
agg_pred = np.sum(ener_est, axis=1)
agg_pred = agg_pred.reshape(-1,1)

RMSE = np.sqrt(
    np.sum( np.square(agg_pred-X_test)
        )/len(X_test))

NRMSE = RMSE/np.mean(X_test)
print("NRMSE :{}".format(NRMSE))

""" compute F1 scores for each load """

f1_score_ = f1_score(y_test, y_pred_test, average=None)
print("f1_score: {} ".format(f1_score_))


average error :0.015576831100549013
NRMSE :0.006579379520490179
f1_score: [0.96955988 0.84717208 0.77383733 0.85626947 0.78347239 0.8987234 ] 
